# extract and transform data


In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
BA_file = 'data/BusinessAnalyst.csv'
busAnal_df = pd.read_csv(BA_file)
busAnal_df.shape
## notes 

(4092, 17)

In [5]:
###  remove "glassdoor est." words from Salary column
##things  go wrong at row 3693
# df.loc[1:3, :]
tophalf_df = busAnal_df.loc[0:3691, :]
tophalf_df.head()
#busAnal_df.shape


,Unnamed: 0,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,0,Business Analyst - Clinical & Logistics Platform,$56K-$102K (Glassdoor est.),Company Overview\n\n\nAt Memorial Sloan Ketter...,3.9,Memorial Sloan-Kettering\n3.9,"New York, NY","New York, NY",10000+ employees,1884,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An...",-1
1,1,1,Business Analyst,$56K-$102K (Glassdoor est.),We are seeking for an energetic and collaborat...,3.8,Paine Schwartz Partners\n3.8,"New York, NY","New York, NY",1 to 50 employees,-1,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1,True
2,2,2,Data Analyst,$56K-$102K (Glassdoor est.),"For more than a decade, Asembia has been worki...",3.6,Asembia\n3.6,"Florham Park, NJ","Florham Park, NJ",501 to 1000 employees,2004,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$5 to $10 million (USD),-1,-1
3,3,3,"Information Security Analyst, Incident Response",$56K-$102K (Glassdoor est.),Job Description Summary\nThe Information Secur...,3.6,BD\n3.6,"Franklin Lakes, NJ","Franklin Lakes, NJ",10000+ employees,1897,Company - Public,Health Care Products Manufacturing,Manufacturing,$10+ billion (USD),"Abbott, Siemens, Baxter",-1
4,4,4,Analyst - FP&A Global Revenue,$56K-$102K (Glassdoor est.),Magnite is the world's largest independent sel...,3.4,Rubicon Project\n3.4,"New York, NY","Los Angeles, CA",201 to 500 employees,2007,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"PubMatic, AppNexus, Index Exchange",True


In [7]:
fixed_tophalf_df = tophalf_df.iloc[:,2:16]
fixed_tophalf_df
#tophalf_df.shape

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Business Analyst - Clinical & Logistics Platform,$56K-$102K (Glassdoor est.),Company Overview\n\n\nAt Memorial Sloan Ketter...,3.9,Memorial Sloan-Kettering\n3.9,"New York, NY","New York, NY",10000+ employees,1884,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),"Mayo Clinic, The Johns Hopkins Hospital, MD An..."
1,Business Analyst,$56K-$102K (Glassdoor est.),We are seeking for an energetic and collaborat...,3.8,Paine Schwartz Partners\n3.8,"New York, NY","New York, NY",1 to 50 employees,-1,Company - Private,Venture Capital & Private Equity,Finance,Unknown / Non-Applicable,-1
2,Data Analyst,$56K-$102K (Glassdoor est.),"For more than a decade, Asembia has been worki...",3.6,Asembia\n3.6,"Florham Park, NJ","Florham Park, NJ",501 to 1000 employees,2004,Company - Private,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$5 to $10 million (USD),-1
3,"Information Security Analyst, Incident Response",$56K-$102K (Glassdoor est.),Job Description Summary\nThe Information Secur...,3.6,BD\n3.6,"Franklin Lakes, NJ","Franklin Lakes, NJ",10000+ employees,1897,Company - Public,Health Care Products Manufacturing,Manufacturing,$10+ billion (USD),"Abbott, Siemens, Baxter"
4,Analyst - FP&A Global Revenue,$56K-$102K (Glassdoor est.),Magnite is the world's largest independent sel...,3.4,Rubicon Project\n3.4,"New York, NY","Los Angeles, CA",201 to 500 employees,2007,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"PubMatic, AppNexus, Index Exchange"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3687,Pega Business Analyst,$58K-$103K (Glassdoor est.),• 5 yrs relevant experience in:- Experience wi...,3.1,Tista Science and Technology Corporation\n3.1,"Austin, TX","Rockville, MD",501 to 1000 employees,2005,Company - Private,Consulting,Business Services,$5 to $10 million (USD),-1
3688,Jr.Business Analyst,$58K-$103K (Glassdoor est.),Job Description\n\nNew businesses are making t...,5.0,"Staffigo Technical Services, LLC\n5.0","Austin, TX","Woodridge, IL",51 to 200 employees,2008,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),-1
3689,Business Process Analyst/Remote 4 mth+ contract,$58K-$103K (Glassdoor est.),Title: Business Process Analyst\nDuration: 4+ ...,-1.0,"Suncap Technology, Inc.","Austin, TX",-1,-1,-1,-1,-1,-1,-1,-1
3690,"AVP, Business System",$58K-$103K (Glassdoor est.),We are looking for a AVP Business System Analy...,3.5,LPL Financial\n3.5,"Austin, TX","Boston, MA",1001 to 5000 employees,1968,Company - Public,Investment Banking & Asset Management,Finance,$2 to $5 billion (USD),"Fidelity Investments, UBS, Morgan Stanley"


In [16]:
#fixed_bottomhalf_df = busAnal_df.loc[3692:, :]
fixed_columnnames = fixed_tophalf_df.columns
print(fixed_columnnames)
fixed_bottomhalf_df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')


Index(['Unnamed: 0', 'index', 'Job Title', 'Salary Estimate',
       'Job Description', 'Rating', 'Company Name', 'Location', 'Headquarters',
       'Size', 'Founded', 'Type of ownership', 'Industry', 'Sector', 'Revenue',
       'Competitors', 'Easy Apply'],
      dtype='object')

In [10]:
fixed_bottomhalf_df = busAnal_df.loc[3692:, :]


In [13]:
# fixed_bottomhalf_df.columns=fixed_columnnames
print(fixed_bottomhalf_df.shape)
fixed_tophalf_df.shape

(400, 17)


(3692, 14)

In [ ]:
#clean_busAnaly_df = fixed_tophalf_df.merge()
#busAnal_df.loc[3692:, :]
clean_busAnaly_df  = pd.concat([fixed_tophalf_df, busAnal_df.loc[3692:, :]], sort=False)
clean_busAnaly_df.shape
#(4092, 17)

In [ ]:
DA_file = 'data/DataAnalyst.csv'
dataAnal_df = pd.read_csv(DA_file)
dataAnal_df

In [ ]:
clean_dataAnaly_df = dataAnal_df.drop(columns = "Unnamed: 0")
clean_dataAnaly_df

In [ ]:
DE_file = 'data/DataEngineer.csv'
dataEng_df = pd.read_csv(DE_file)
#no changes needed to this dataset, so setting name to "clean"
clean_dataEng_df = dataEng_df

In [ ]:
DS_file = 'data/DataScientist.csv'
dataSci_df = pd.read_csv(DS_file)
dataSci_df

In [ ]:
clean_dataSci_df = dataSci_df.drop(columns = ["Unnamed: 0","index"])
clean_dataSci_df

# next comments do here.

In [ ]:
##  todo list
# get rid of "glassdoor" words in salary
# get rid of weird characters in company name
# join everything into giant table/df

In [ ]:
# add job category to each DF
clean_dataSci_df["Job Category"] = "Data Science"
clean_dataEng_df["Job Category"] = "Data Engineer"
clean_dataAnaly_df["Job Category"] = "Data Analyst"
clean_busAnaly_df["Job Category"] = "Business Analyst"

In [ ]:
print(clean_dataEng_df.columns)
print(clean_dataAnaly_df.columns)
print(clean_busAnaly_df.columns)
print(clean_dataSci_df.columns)

In [ ]:
#congif.py document set user = <type your user name>, password = <type your password>, port = 5432, host = “localhost”.

In [ ]:
#load to database
from config import user, password, host, port

In [ ]:
db = 'NAME OF DATABASE GOES HERE'
rds_connection_string = f"{user}:{password}@{host}:{port}/{db}"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# looking at tables in db
engine.table_names()

In [ ]:
# loading df into db table
DataSci_df.to_sql(name='NAME OF TABLE GOES HERE', con=engine, if_exists='append', index=False)